In [63]:
!pip install confluent_kafka sseclient

In [64]:
from confluent_kafka import SerializingProducer, DeserializingConsumer
from confluent_kafka.serialization import StringSerializer, StringDeserializer
from confluent_kafka.admin import AdminClient, NewTopic
from uuid import uuid4
import sys, random

brokers = "kafka1:9092,kafka2:9093"

In [65]:
pconf = {
    'bootstrap.servers': brokers,
    'partitioner': 'murmur2_random',
    'key.serializer': StringSerializer('utf_8'),
    'value.serializer':  StringSerializer('utf_8')
}

In [66]:
p = SerializingProducer(pconf)

In [67]:
stocks = "sample.csv"

In [68]:
from datetime import datetime  
  #2023-10-13T08:16:13Z
def construct_stock(row):
    time_stamp = time.time()
    date_time = datetime.fromtimestamp(time_stamp)
    str_date_time = date_time.strftime("%Y-%m-%dT%H:%M:%SZ") #"%d-%m-%Y, %H:%M:%S"
    stock = {"TODO": row[6],
             "TODO": float(row[2]),
             "timestamp":str_date_time
             }
    return stock

In [69]:
import csv
import json
from datetime import datetime

def construct_stock(row):
    try:
        return {
            "license": row[0],  
            "total_amount": float(row[1]),
            "timestamp": datetime.fromtimestamp(int(row[5])).strftime("%Y-%m-%dT%H:%M:%SZ")
        }
    except ValueError:
        print(f"Skipping row with non-numeric value: {row}")
        return None
  
stocks = "sample.csv"  
n = 0

with open(stocks) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    header = next(csv_reader) # read the header row
    for row in csv_reader:
        n = n + 1
        stock = construct_stock(row)
        if stock is not None:
            p.produce('stock', value=json.dumps(stock))
    

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



Skipping row with non-numeric value: ['AC685D6E8D5F37E36209988E510DADEE', '0BB61C85B29B80015D67F4445466669C', 'VTS', '1', '', '2013-01-13 19:09:00', '2013-01-13 19:22:00', '1', '780', '3.15', '-73.961639', '40.764557', '-74.001266', '40.755424']
Skipping row with non-numeric value: ['FD656CFF77011E79FD49FE4DAB7D1A1C', '7EA632323D0353F462566B217E033F84', 'VTS', '1', '', '2013-01-13 19:19:00', '2013-01-13 19:24:00', '1', '300', '1.15', '-73.988838', '40.745701', '-74.002617', '40.740063']
Skipping row with non-numeric value: ['04C271CE9B4F096FD7EC1E3733C01579', '905351B4B6B0090F05608E53EB948BBC', 'VTS', '1', '', '2013-01-13 18:50:00', '2013-01-13 18:57:00', '6', '420', '1.40', '-73.983528', '40.688831', '-73.982834', '40.675747']
Skipping row with non-numeric value: ['F6BB15B0F863D58E24EAC1D99AA7230D', 'DE71A0CE2C79C14DE76DD394D3CBBBF3', 'VTS', '1', '', '2013-01-13 19:00:00', '2013-01-13 19:26:00', '1', '1560', '15.40', '-73.870354', '40.77359', '-74.01432', '40.704266']
Skipping row wit

### Use only if you need to purge all the messages in the queue

In [70]:
brokers = "kafka1:9092,kafka2:9093"

admin_client = AdminClient({"bootstrap.servers":brokers})
admin_client.delete_topics(topics=['stock'])

{'stock': <Future at 0x7fa6705bee10 state=running>}